In [1]:
import AlGDock.Nanopore
from AlGDock.Nanopore import *

self = AlGDock.Nanopore.Simulation(\
  ligand_tarball='prmtopcrd/ligand.tar.gz', \
  ligand_database='ligand.db', \
  forcefield='prmtopcrd/gaff.dat', \
  frcmodList=['ligand.frcmod'], \
  ligand_prmtop='ligand.prmtop', \
  grid_LJr='grids/LJr.nc', \
  starting_conf='prmtopcrd/anchor_and_grow_scored.mol2', \
  ef=1.0E8, \
  max_trials=10000, \
  report_interval=100)

  grid type LJa not found
  grid type ELE not found


         /Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db,
         using first one
/Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db
/Users/dminh/Installers/AlGDock-0.0.1/Nanopore_Example/ligand.db


In [2]:
self.run()

Simulating motion from 0.20000 to 1.80000
Trial      0, COM: [0.48641011897147823, 0.5195822239253441, 0.2274598866830508]


/Users/dminh/Applications/miniconda2/envs/algdock/lib/python2.7/site-packages/simtk/openmm/app/internal/amber_file_parser.py:1010: UserWarning: Non-optimal GB parameters detected for GB model OBC2
  'model %s' % gbmodel)


Trial    100, COM: [0.5018127513637335, 0.38280997947467105, 0.5289558304543043]
Trial    200, COM: [0.7028397194289955, 0.46225581754458617, 0.6137105982961111]
Trial    300, COM: [0.6854178086419123, 0.41111290528306177, 0.6382155870254187]
Trial    400, COM: [0.6772097200302113, 0.3878595517087754, 0.6261963544390063]
Trial    500, COM: [0.5468343068302401, 0.3921164542654682, 0.793468958583295]
Trial    600, COM: [0.5184887430310394, 0.5146105893848353, 0.9405338672561881]
Trial    700, COM: [0.666105288421152, 0.5187444172041633, 0.9671476436991225]
Trial    800, COM: [0.5747669896711305, 0.4553522638827878, 1.1123100070897498]
Trial    900, COM: [0.5197198713456933, 0.5375225651740471, 1.4543231266393373]
Trial   1000, COM: [0.4988038317545555, 0.6351729493038265, 1.6524294639424875]
Trial   1100, COM: [0.4166511058153393, 0.772662823251289, 1.6583318882075226]
Trial   1200, COM: [0.3591009912996962, 0.6980928057824677, 1.7289306424819062]
Number of trials before pore crossing:  

In [3]:
self.view_trajectory()